In [369]:
import pandas as pd
import numpy as np 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import nltk
nltk.download('punkt')#
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')#
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords') #
import sys  
!{sys.executable} -m pip install pyspellchecker 
from spellchecker import SpellChecker 

[nltk_data] Downloading package punkt to /Users/Tobias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Tobias/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Tobias/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Tobias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [370]:
train = pd.read_csv('test.csv',encoding ="utf-8" )
#pd.set_option('display.max_colwidth', -1)

In [371]:
#train.drop(["id"], axis=1, inplace=True)

In [372]:
train["text"]=train["text"].str.lower()

In [373]:
trainS = train
trainS.head(20)

,id,keyword,location,text
0,0,NaN,NaN,just happened a terrible car crash
1,2,NaN,NaN,"heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, i cannot save them all"
3,9,NaN,NaN,apocalypse lighting. #spokane #wildfires
4,11,NaN,NaN,typhoon soudelor kills 28 in china and taiwan
5,12,NaN,NaN,we're shaking...it's an earthquake
6,21,NaN,NaN,"they'd probably still show more life than arsenal did yesterday, eh? eh?"
7,22,NaN,NaN,hey! how are you?
8,27,NaN,NaN,what a nice hat?
9,29,NaN,NaN,fuck off!


In [374]:
Contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"gonna": "going to",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [375]:
def data_text_preprocess(total_text):
        
        total_text = re.sub('http\S+',' ', str(total_text))
        total_test = re.sub('https\S+',' ', str(total_text))
        total_text = re.sub('\@\S+',' ', str(total_text))
        total_text = re.sub('\#',' ', str(total_text))
        total_text = re.sub('[^a-zA-Z\n]', ' ', str(total_text))
        total_text = re.sub(r" \B'\b|\b'\B", " ", str(total_text))
        
       
        return total_text

In [376]:
trainS['text'] = trainS['text'].map(data_text_preprocess)
trainS.head(20)

,id,keyword,location,text
0,0,NaN,NaN,just happened a terrible car crash
1,2,NaN,NaN,heard about earthquake is different cities stay safe everyone
2,3,NaN,NaN,there is a forest fire at spot pond geese are fleeing across the street i cannot save them all
3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan
5,12,NaN,NaN,we re shaking it s an earthquake
6,21,NaN,NaN,they d probably still show more life than arsenal did yesterday eh eh
7,22,NaN,NaN,hey how are you
8,27,NaN,NaN,what a nice hat
9,29,NaN,NaN,fuck off


In [377]:
 def decontraction(total_text):
        
        for i in Contractions:
            total_text = total_text.replace( i, Contractions.get(i))
        
        return total_text

In [378]:
trainS['text'] = trainS['text'].map(decontraction)
trainS.head(20)

,id,keyword,location,text
0,0,NaN,NaN,just happened a terrible car crash
1,2,NaN,NaN,heard about earthquake is different cities stay safe everyone
2,3,NaN,NaN,there is a forest fire at spot pond geese are fleeing across the street i cannot save them all
3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan
5,12,NaN,NaN,we re shaking it s an earthquake
6,21,NaN,NaN,they d probably still show more life than arsenal did yesterday eh eh
7,22,NaN,NaN,hey how are you
8,27,NaN,NaN,what a nice hat
9,29,NaN,NaN,fuck off


In [379]:
def get_wordnet_pos(word):
   
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [380]:
lemmatizer = WordNetLemmatizer()

def lemmatization1(total_text):
    total_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(total_text)]

    return total_text

In [381]:
trainS['text'] = trainS['text'].map(lemmatization1)
trainS.head(20)

,id,keyword,location,text
0,0,NaN,NaN,"[just, happen, a, terrible, car, crash]"
1,2,NaN,NaN,"[heard, about, earthquake, be, different, city, stay, safe, everyone]"
2,3,NaN,NaN,"[there, be, a, forest, fire, at, spot, pond, geese, be, flee, across, the, street, i, can, not, save, them, all]"
3,9,NaN,NaN,"[apocalypse, light, spokane, wildfire]"
4,11,NaN,NaN,"[typhoon, soudelor, kill, in, china, and, taiwan]"
5,12,NaN,NaN,"[we, re, shake, it, s, an, earthquake]"
6,21,NaN,NaN,"[they, d, probably, still, show, more, life, than, arsenal, do, yesterday, eh, eh]"
7,22,NaN,NaN,"[hey, how, be, you]"
8,27,NaN,NaN,"[what, a, nice, hat]"
9,29,NaN,NaN,"[fuck, off]"


In [382]:
stop_words = set(stopwords.words('english'))


In [383]:
def remove_stopwords(total_text):
    
    filtered_word_list = total_text[:]
    for word in total_text:
        if word in stop_words: 
            filtered_word_list.remove(word)
    
    #filtered_sentence = [w for w in total_text if not w in stop_words] 
    #filtered_sentence= []
    
    #for w in total_text: 
      #  if w not in stop_words: 
       #     filtered_sentence.append(w) 

    return filtered_word_list

In [384]:
trainS['text'] = trainS['text'].map(remove_stopwords)
trainSR = trainS
trainSR.head(20)

,id,keyword,location,text
0,0,NaN,NaN,"[happen, terrible, car, crash]"
1,2,NaN,NaN,"[heard, earthquake, different, city, stay, safe, everyone]"
2,3,NaN,NaN,"[forest, fire, spot, pond, geese, flee, across, street, save]"
3,9,NaN,NaN,"[apocalypse, light, spokane, wildfire]"
4,11,NaN,NaN,"[typhoon, soudelor, kill, china, taiwan]"
5,12,NaN,NaN,"[shake, earthquake]"
6,21,NaN,NaN,"[probably, still, show, life, arsenal, yesterday, eh, eh]"
7,22,NaN,NaN,[hey]
8,27,NaN,NaN,"[nice, hat]"
9,29,NaN,NaN,[fuck]


In [385]:
def removesinglelettertoken(total_text):
    emptylist=[]
    for w in total_text:
        if len(w)>1:
            emptylist.append(w)
    return emptylist

In [386]:
trainSR['text'] = trainSR['text'].map(removesinglelettertoken)

trainSR.head(20)

,id,keyword,location,text
0,0,NaN,NaN,"[happen, terrible, car, crash]"
1,2,NaN,NaN,"[heard, earthquake, different, city, stay, safe, everyone]"
2,3,NaN,NaN,"[forest, fire, spot, pond, geese, flee, across, street, save]"
3,9,NaN,NaN,"[apocalypse, light, spokane, wildfire]"
4,11,NaN,NaN,"[typhoon, soudelor, kill, china, taiwan]"
5,12,NaN,NaN,"[shake, earthquake]"
6,21,NaN,NaN,"[probably, still, show, life, arsenal, yesterday, eh, eh]"
7,22,NaN,NaN,[hey]
8,27,NaN,NaN,"[nice, hat]"
9,29,NaN,NaN,[fuck]


In [387]:
spell = SpellChecker()

def spellchecker(total_text):
    for w in total_text: 
        spell.correction(w) 
    return total_text

In [388]:
#trainSR['text'] = trainSR['text'].map(spellchecker)

#trainSR.head(20)

In [389]:
# Save to csv
trainSR.to_csv("testclean.csv", index=False, encoding = 'utf-8')